In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
file_path = "citation_network.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)
#df = pd.read_csv(file_path,nrows=1000)

# sửa không tìm ra

In [ ]:
import pandas as pd
import numpy as np

# Initialize an empty list to store the results
results = []

# Create a mapping of references to their corresponding metadata
reference_metadata = df.set_index('id')[['year', 'n_citation', 'doc_type']].to_dict(orient='index')

# Process each row in the DataFrame
for index, row in df.iterrows():
    paper_id = row['id']
    references = row['references']
    
    if pd.isna(references):
        # If references is NaN, add a row with ref -1 and corresponding metadata as NaN
        results.append({'id': paper_id, 'ref': -1, 'year': np.nan, 'n_citation': np.nan, 'doc_type': np.nan})
    else:
        # Split references by semicolon and add to results
        refs = references.split(';')
        for ref in refs:
            ref = ref.strip()  # Remove any leading or trailing whitespace
            # Get the metadata for the reference, defaulting to NaN if not found
            metadata = reference_metadata.get(int(ref), {'year': np.nan, 'n_citation': np.nan, 'doc_type': np.nan})
            results.append({
                'id': paper_id,
                'ref': ref,
                'year': metadata['year'],
                'n_citation': metadata['n_citation'],
                'doc_type': metadata['doc_type']
            })

# Create a new DataFrame from the results and reset index to create a default index
results_df = pd.DataFrame(results).reset_index(drop=True)


In [ ]:
# New DataFrame to hold the results
result = []

# Processing each row
for index, row in df.iterrows():
    # Check if references is a string
    if isinstance(row['references'], str):
        ref_list = row['references'].split(';')
        for ref in ref_list:
            if ref.strip():  # If ref is not empty
                result.append({'id': row['id'], 'ref': ref.strip()})
    else:
        # Handle NaN by setting ref to -1
        result.append({'id': row['id'], 'ref': -1})

# Creating the result DataFrame
result_df = pd.DataFrame(result)

# Display the resulting DataFrame
print(result_df)

In [ ]:
# Function to find a specific ref
def find_ref(ref_value):
    # Filter the DataFrame for the specified ref
    filtered_df = result_df[result_df['ref'] == ref_value]
    return filtered_df

# Example usage
search_ref = '2005687710'  # Replace with the ref you want to find
found_refs = find_ref(search_ref)

# Display the results
if not found_refs.empty:
    print("Found references:")
    print(found_refs)
else:
    print(f"No references found for: {search_ref}")

In [ ]:
# Function to add the year based on matching ref with original DataFrame's id
def add_year_to_results(result_df, original_df):
    # Merge with the original DataFrame based on ref and id
    # Create a temporary DataFrame for the references
    refs_df = original_df[['id', 'year']].rename(columns={'id': 'ref'})
    
    # Perform a left join to add the year information
    merged_df = result_df.merge(refs_df, on='ref', how='left')
    
    # If ref was -1 (NaN), set year to NaN as well
    merged_df['year'].fillna(-1, inplace=True)  # Set NaN to -1 if desired
    
    return merged_df

# Add year to result_df
final_result_df = add_year_to_results(result_df, df)


In [ ]:
# Function to find a specific ref
def find_ref_final_result_df(ref_value):
    # Filter the DataFrame for the specified ref
    filtered_df = final_result_df[final_result_df['ref'] == ref_value]
    return filtered_df

# Example usage
search_ref = '2005687710'  # Replace with the ref you want to find
found_refs = find_ref_final_result_df(search_ref)

# Display the results
if not found_refs.empty:
    print("Found references:")
    print(found_refs)
else:
    print(f"No references found for: {search_ref}")

In [ ]:
def add_year_and_year_ref(result_df, original_df):
    # Create a temporary DataFrame for the references from the original DataFrame
    refs_df = original_df[['id', 'year']].rename(columns={'id': 'ref'})
    
    # Merge with the result DataFrame based on the ref
    merged_df = result_df.merge(refs_df, on='ref', how='left', suffixes=('', '_original'))

    # Create new columns for year and year_ref
    merged_df['year'] = merged_df['year_original'].fillna(0)
    merged_df['year_ref'] = merged_df['year'].where(merged_df['year'] != 0, 0)

    # Drop the temporary column used for merging
    merged_df.drop(columns=['year_original'], inplace=True)

    return merged_df

# Example usage
# final_result_df should already be defined with appropriate data
final_result_df = add_year_and_year_ref(final_result_df, df)
final_result_df

n_citation_ref

In [264]:
def add_additional_columns(result_df, original_df):
    # Create a temporary DataFrame for the references from the original DataFrame
    refs_df = original_df[['id', 'year', 'n_citation', 'doc_type']].rename(columns={'id': 'ref'})
    
    # Merge with the result DataFrame based on the ref
    merged_df = result_df.merge(refs_df, on='ref', how='left', suffixes=('', '_original'))

    # Create new columns for year, year_ref, n_citation_ref, and doc_type
    merged_df['year'] = merged_df['year'].fillna(0)
    merged_df['year_ref'] = merged_df['year'].where(merged_df['year'] != 0, 0)

    # Rename n_citation to n_citation_ref and fill with 0 if not present
    merged_df['n_citation_ref'] = merged_df['n_citation'].fillna(0)
    merged_df['doc_type'] = merged_df['doc_type'].fillna("none")

    # Add a new column for the original id's n_citation from original_df
    merged_df = merged_df.merge(original_df[['id', 'n_citation']], on='id', how='left', suffixes=('', '_original_n'))
    merged_df['n_citation_id'] = merged_df['n_citation_original_n'].fillna(0)

    # # Drop the temporary columns used for merging, if desired
    # merged_df.drop(columns=['n_citation_original_n'], inplace=True)

    return merged_df

# Example usage
# final_result_df should already be defined with appropriate data
final_result_df_fix_id_ref = add_additional_columns(final_result_df, df)


In [ ]:
final_result_df_fix_id_ref

In [ ]:
# Function to find a specific ref
def find_final_result_df_fix_id_ref(ref_value):
    # Filter the DataFrame for the specified ref
    filtered_df = final_result_df_fix_id_ref[final_result_df_fix_id_ref['ref'] == ref_value]
    return filtered_df

# Example usage
search_ref = '2005687710'  # Replace with the ref you want to find
found_refs = find_final_result_df_fix_id_ref(search_ref)

# Display the results
if not found_refs.empty:
    print("Found references:")
    print(found_refs)
else:
    print(f"No references found for: {search_ref}")

In [267]:
# drop collumn n_citation_id from copy_result_df
final_result_df_fix_id_ref.drop('year_original', axis=1, inplace=True)
final_result_df_fix_id_ref.drop('n_citation', axis=1, inplace=True)
final_result_df_fix_id_ref.drop('n_citation_id', axis=1, inplace=True)
final_result_df_fix_id_ref.rename(columns={'n_citation_original_n': 'n_citation'}, inplace=True)

In [ ]:
final_result_df_fix_id_ref

In [271]:
result_df=final_result_df_fix_id_ref

In [272]:
final_result_df = result_df

In [273]:
# Fill missing values
result_df['year'] = result_df['year'].fillna(0)
result_df['n_citation'] = result_df['n_citation'].fillna(0)
result_df['doc_type'] = result_df['doc_type'].fillna('none')
result_df['ref'] = result_df['ref'].replace(-1,0)

In [ ]:
result_df 

In [275]:
result_df.to_csv('id_ref_year_nCitation_docType.csv')

saii

In [ ]:
unique_doc_type = result_df['doc_type'].unique()
unique_doc_type

In [ ]:
# Finding min and max year, ignoring the default '0' year for id 1388
valid_years = result_df[result_df['year_ref'] > 0]['year_ref']  # Filter out years that are 0

min_year = valid_years.min() if not valid_years.empty else None
max_year = valid_years.max() if not valid_years.empty else None

# Adjust the minimum year by subtracting 0 (no change)
adjusted_min_year = min_year - 0 if min_year is not None else None

# Display the results
print("Minimum Year (adjusted):", adjusted_min_year)
print("Maximum Year:", max_year)

In [ ]:

# Convert n_citation to numeric, coercing errors to NaN
result_df['n_citation'] = pd.to_numeric(result_df['n_citation'], errors='coerce')

# Fill NaN values with 0
result_df['n_citation'] = result_df['n_citation'].fillna(0).astype(int)

# Convert n_citation to numeric, coercing errors to NaN
result_df['n_citation_ref'] = pd.to_numeric(result_df['n_citation_ref'], errors='coerce')

# Fill NaN values with 0
result_df['n_citation_ref'] = result_df['n_citation_ref'].fillna(0).astype(int)
# Display the resulting DataFrame
print("Updated DataFrame with n_citation as int:")
print(result_df)

In [ ]:
# Assuming final_result_df is already defined and contains the necessary columns

# Filter values greater than 0 for both columns
valid_n_citation_ref = final_result_df[final_result_df['n_citation_ref'] > 0]['n_citation_ref']
valid_n_citation_id = final_result_df[final_result_df['n_citation'] > 0]['n_citation']

# Find min and max for n_citation_ref
min_n_citation_ref = valid_n_citation_ref.min() if not valid_n_citation_ref.empty else None
max_n_citation_ref = valid_n_citation_ref.max() if not valid_n_citation_ref.empty else None

# Find min and max for n_citation_id
min_n_citation_id = valid_n_citation_id.min() if not valid_n_citation_id.empty else None
max_n_citation_id = valid_n_citation_id.max() if not valid_n_citation_id.empty else None

# Display results
print("n_citation_ref - Minimum (excluding 0):", min_n_citation_ref)
print("n_citation_ref - Maximum (excluding 0):", max_n_citation_ref)
print("n_citation_id - Minimum (excluding 0):", min_n_citation_id)
print("n_citation_id - Maximum (excluding 0):", max_n_citation_id)


In [ ]:
result_df

In [ ]:
# Function to find a specific ref
def find_result_df(ref_value):
    # Filter the DataFrame for the specified ref
    filtered_df = result_df[result_df['ref'] == ref_value]
    return filtered_df

# Example usage
search_ref = '2005687710'  # Replace with the ref you want to find
found_refs = find_result_df(search_ref)

# Display the results
if not found_refs.empty:
    print("Found references:")
    print(found_refs)
else:
    print(f"No references found for: {search_ref}")

In [281]:
result_df.to_csv('id_ref_year_nCitation_docType.csv')

# cập nhật n_citation

In [ ]:
copy_result_df = pd.read_csv('id_ref_year_nCitation_docType.csv')

In [ ]:
copy_result_df = result_df
copy_result_df

In [207]:
result_df = copy_result_df

In [284]:
# Count occurrences of each id in the ref column
id_counts = copy_result_df['ref'].value_counts()
# Map the counts back to a new column n_citation_id in the original DataFrame
copy_result_df['n_citation_id'] = copy_result_df['id'].map(id_counts).fillna(0).astype(int)

In [ ]:
# Function to find a specific ref
def find_result_df(ref_value):
    # Filter the DataFrame for the specified ref
    filtered_df = result_df[result_df['ref'] == ref_value]
    return filtered_df

# Example usage
search_ref = '2005687710'  # Replace with the ref you want to find
found_refs = find_result_df(search_ref)

# Display the results
if not found_refs.empty:
    print("Found references:")
    print(found_refs)
else:
    print(f"No references found for: {search_ref}")

In [ ]:
import pandas as pd
from tqdm import tqdm

def update_n_citation_id_based_on_ref_count_with_progress(result_df):
    # Create a copy of the result DataFrame to track changes
    updated_df = result_df.copy()

    # Initialize a list to track changed IDs
    changed_ids = []

    # Count occurrences of each ref in the result DataFrame
    ref_counts = updated_df['ref'].value_counts()

    # Use tqdm to display progress while looping through each row in the updated DataFrame
    for index, row in tqdm(updated_df.iterrows(), total=updated_df.shape[0], desc="Updating n_citation_id"):
        # Get the current ref
        ref = row['ref']
        
        # Check if the ref exists in the ref_counts
        if ref in ref_counts:
            # Update n_citation_id with the count of that ref
            updated_df.at[index, 'n_citation_id'] = ref_counts[ref]

            # Add the ID to the changed list if it has been updated
            if row['n_citation_id'] != ref_counts[ref]:
                changed_ids.append(row['id'])

    return updated_df, changed_ids

# Example usage
# final_result_df should already be defined with appropriate data
final_result_df, updated_ids = update_n_citation_id_based_on_ref_count_with_progress(final_result_df)

# Print the changed IDs7
if updated_ids:
    print("Updated n_citation_id for IDs:", updated_ids)
else:
    print("No IDs were updated.")


In [ ]:
final_result_df

In [ ]:
# Find rows where n_citation = 0 and n_citation_id != 0
dif_df = copy_result_df[(copy_result_df['n_citation_ref'] == 0) & (copy_result_df['n_citation_id'] != 0)]
dif_df

In [244]:
# Check for rows where n_citation = 0 and n_citation_id != 0
condition = (copy_result_df['n_citation_ref'] == 0) & (copy_result_df['n_citation_id'] != 0)

# Update n_citation where the condition is met
copy_result_df.loc[condition, 'n_citation_ref'] = copy_result_df.loc[condition, 'n_citation_id']

In [ ]:
copy_result_df

In [247]:
# drop collumn n_citation_id from copy_result_df
copy_result_df.drop('n_citation_id', axis=1, inplace=True)

In [ ]:
copy_result_df

In [216]:
copy_result_df.to_csv('update_nCitation.csv')

In [248]:
def update_year_with_original_id(result_df, original_df):
    # Create a temporary DataFrame for the original DataFrame containing id and year
    year_df = original_df[['id', 'year']]

    # Merge the original year data into the result DataFrame based on id
    updated_df = result_df.merge(year_df, on='id', how='left', suffixes=('', '_original'))

    # Update the year column in result_df with the year from original_df
    updated_df['year'] = updated_df['year_original'].fillna(updated_df['year'])

    # Drop the temporary year_original column
    updated_df.drop(columns=['year_original'], inplace=True)

    return updated_df

# Example usage
# final_result_df should already be defined with appropriate data
final_final = update_year_with_original_id(copy_result_df, df)

In [ ]:
final_final

In [250]:
final_final.to_csv('final.csv')

In [ ]:
# Function to calculate points based on the conditions
def calculate_points(row):
    if row['n_citation_ref'] > 0:
        if row['year_ref'] == 0:  # If year is 0, calculate points as n_citation * 2
            points = row['n_citation_ref'] * 10
        else:  # If year is not 0, use the original calculation
            points = (row['year_ref'] - 1800) * 10 + (row['n_citation_ref'] * 10)
    else:  # No refs
        points = row['n_citation_ref'] * 10
    return points
# Calculate points and create new column
final_final['Point'] = final_final.apply(calculate_points, axis=1)
final_final

In [252]:
# Function to calculate the point
def calculate_point(row):
    if row['year_ref'] != 0:
        return (row['year_ref'] - 1800) * 10 + row['n_citation_ref'] * 10 + row['n_citation'] * 2
    else:
        return row['n_citation_ref'] * 10 + row['n_citation'] * 2

# Applying the function to calculate points
final_final['Point'] = final_final.apply(calculate_point, axis=1)

In [ ]:
#Counting the number of rows where Point equals 0
count_point_zero = final_final[final_final['Point'] == 0].shape[0]

# Display the result
print(f"Number of rows with Point = 0: {count_point_zero}")

In [ ]:
final_final['Point'].max()
